In [ ]:
!pip install aladdinsdk
!pip list | grep sdk

# Initial configuration setup

### Setup a configuration file.

- Open a terminal and run `aladdinsdk-cli`, and follow the steps to create a configuration file.
- Per the final instruction in the CLI, set the ASDK_USER_CONFIG_FILE environment variable **before** importing aladdinsdk

In [ ]:
import os
os.environ["ASDK_USER_CONFIG_FILE"] = '<path to config file>'

# Set environment variable prior to import, either in code / terminal
import aladdinsdk

Utility to validate configuration is picked correctly:

In [ ]:
from aladdinsdk.config import print_current_user_config
print_current_user_config()

----- 

# ADC Connectivity


In [ ]:
import aladdinsdk
from aladdinsdk.adc.client import ADCClient

adc_client = ADCClient()

df = adc_client.query_sql('SELECT * FROM CASH_ENTRY_V2 LIMIT 10')
df

**Note:**
- Configuration files are optional, each config can be overriden in-line
- Environment variables take precedence, and may be used to set unmodifiable configurations

-----

# Common Dev Utilities

## Batch Processing

In [ ]:
import aladdinsdk
from aladdinsdk.common.batch.action import SDKActionBuffer, SDKAction
from aladdinsdk.api.client import AladdinAPI

action_buffer = SDKActionBuffer()

adc_client = ADCClient()
adc_query_action = SDKAction(adc_client.query_sql, 'SELECT * FROM CASH_ENTRY_V2 LIMIT 10')
action_buffer.append_action(adc_query_action)

adc_query_action_2 = SDKAction(adc_client.query_sql, 'SELECT * FROM CASH_ENTRY_V2 LIMIT 100')
action_buffer.append_action(adc_query_action_2)


# action_buffer.run_sequential()
action_buffer.run_parallel()

response_map = action_buffer.get_response_map()
for action_uid in response_map:
    res = response_map[action_uid]
    print(f"result for action {action_uid}: {type(res)}")

In [ ]:
# Batch Processing example with print statements to understand the flow

from aladdinsdk.common.batch.action import SDKActionBuffer, SDKAction
import time

def func1(num, sleep_time):
    from datetime import datetime
    starttime = datetime.now()
    print(f"start {num}-{sleep_time}")
    time.sleep(sleep_time)
    endtime = datetime.now()
    print(f"{endtime - starttime} end {num}-{sleep_time}")
    return num

# Action buffer setup
action_buffer = SDKActionBuffer()
action_buffer.append_action(SDKAction(func1, 1, 2))
action_buffer.append_action(SDKAction(func1, 2, 3))
action_buffer.append_action(SDKAction(func1, 3, 8))
action_buffer.append_action(SDKAction(func1, 4, 8))
action_buffer.append_action(SDKAction(func1, 5, 8))
action_buffer.append_action(SDKAction(func1, 6, 8))
action_buffer.append_action(SDKAction(func1, 7, 8))
action_buffer.append_action(SDKAction(func1, 8, 10))

# action_buffer.run_sequential()  # optional arg: interval: to pause between actions
action_buffer.run_parallel()  # optional arg: max_workers to limit worker threads

# for action in action_buffer.actions:
#     print(f"result for action {action.uid} result {action.result} error {action.error}")
for result in action_buffer.get_response_map().values():
    print(f"result for action {result}")



------------

### References:
- Engineering blog - https://engineering.blackrock.com/open-sourcing-the-aladdinsdk-empower-python-developers-with-a-quantitative-edge-7f63376061e6
- Open source project - https://github.com/blackrock/aladdinsdk
    - https://pypi.org/project/aladdinsdk/
- Internal getting started guide - https://webster.bfm.com/apps/studio/knowledge-center/get-started-with-aladdinsdk-blk